## Stemmen voor alles behalve de stem
### Een visuele analyse van het stemgedrag bij het Eurovisie Songfestival

Ieder jaar, zo halverwege mei, verbindt het Eurovisie Songfestival de muziekliefhebbers over heel Europa. Het festival werd voor het eerst gehouden in 1956 met slechts zeven landen. Vandaag de dag doen er rond de veertig landen mee. Ieder land treedt op met een zelfgeschreven nummer, waarna er via stemming van een jury en televoting een winnaar wordt bepaald. Ieder land reikt punten uit waarbij 50 procent via televoting en 50 procent via een jury bepaald worden. Tot 2016 gaf een land 1 tot en met 8, 10 en 12 punten aan andere landen. Vanaf 2016 werden dit twee sets, één namens de jury en één namens televoting. Onderstaand figuur geeft de punten weer die de landen gehaald hebben over de jaren 

In [48]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import csv

color_blind_red, color_blind_green, color_blind_blue = '#d95f02', '#1b9e77', '#7570b3'

In [49]:
# Importeer het dataframe met de punten die gegeven zijn per songfestival
euro_df = pd.read_csv('eurovision_song_contest_1975_2019.csv')
euro_df.columns=[column.strip() for column in euro_df.columns]


In [50]:
# Kaart van de ontwikkeling van het gemiddelde van de punten in de finale per land
def total_per_year(dataframe):
    """
    Berekent de totale hoeveelheid punten die een land per jaar in de finale van het eurovisie
    van de vakjury heeft gekregen.
    """

    # Maak een lege dictionary aan voor het opslaan van de data
    total_year = {}
    
    # Itereer over de dataframe met gegeven punten
    for index, row in dataframe.iterrows():

        # Controleer of er een finale gezongen werd en of de punten gegeven zijn door
        # de vakjury
        if row[1] == 'f' and row[3] == 'J':

            # Controleer of het land al in het dictionary staat
            if row[5] in total_year:

                # Controleer of het jaar al bij het land staat
                if row[0] in total_year[row[5]]:

                    # Als het jaar al aanwezig is bij het land, worden de punten opgeteld
                    total_year[row[5]][row[0]] += row[6]

                # Als het jaar nog niet aanwezig is wordt het jaar toegevoegd aan het dictionary
                # van het land
                else:
                    total_year[row[5]].update({row[0]: row[6]})

            # Als het land nog niet in het dictionary staat, wordt een nieuwe key aangemaakt met
            # daarin weer een dictionary van het jaar en het aantal punten dat behaald is
            else:
                total_year.update({row[5]: {row[0]:row[6]}})

    # Maak een dataframe om het dictionary in te kunnen zetten
    total_per_year_df = pd.DataFrame({'Country':[], 'Year':[], 'Points':[]})

    # Voeg alle elementen van het dictionary per land toe aan het dataframe
    for country, euro_df in total_year.items():
        for year, score in euro_df.items():
            total_per_year_df.loc[len(total_per_year_df)] = [country, year, score]

    return total_per_year_df

def development_of_mean(dataframe):
    """
    Berekent de verandering van het gemiddelde totale aantal punten dat een land per jaar heeft gekregen.
    Elk jaar wordt dus het gemiddelde geupdate naar het nieuwe aantal punten dat is behaald, tenzij
    er niet is deelgenomen aan de finale. In dat geval wordt hetzelfde gemiddelde nog eens toegevoegd
    om gaten in de uiteindelijke grafiek te voorkomen. De functie vult ook de waardes aan tot het jaar
    2019, zodat de grafiek bij het eindpunt van de animatie alle uiteindelijke gemiddeldes bevat.
    """

    # Creëer een dataframe met de totale aantal punten per land per jaar
    total_per_year_df = total_per_year(dataframe)

    # Maak een dictionary met het eerste land, jaar en punten 
    mean_per_year = {total_per_year_df.iloc[0,0] : {total_per_year_df.iloc[0,1]: total_per_year_df.iloc[0,2]}}

    # Maak counters om de gemiddelden mee te kunnen berekenen
    total_years = 0
    total_points = total_per_year_df.iloc[0,2]

    # Loop over het dataframe met totale aantal punten, sla de eerste rij over
    for index, row in total_per_year_df[1:].iterrows():

        # Controleer of het land hetzelfde is als in de rij ervoor
        if row[0] == total_per_year_df.iloc[index - 1, 0]:

            # Als het land hetzelfde is, controleer of het jaar maar één verschilt
            # Zo niet, vul dan de missende jaren aan met het bestaande gemmiddelde
            if row[1] != (total_per_year_df.iloc[index - 1, 1] + 1):
                for year in range(total_per_year_df.iloc[index - 1, 1] + 1, row[1]):
                    mean_per_year[row[0]].update({year : (total_points / total_years)})

            # Vul het dict aan met het jaar en het nieuwe gemiddelde
            total_points += row[2]
            total_years += 1
            mean_per_year[row[0]].update({row[1] : (total_points / total_years)})        

        else:

            # Als het land niet hetzelfde is als de rij ervoor, controleer of het laatst aangevulde jaar 2019 is
            # Zo niet, vul dan het dict aan met hetzelfde gemiddelde tot 2019
            if total_per_year_df.iloc[index - 1, 1] != 2019:
                for year in range(total_per_year_df.iloc[index - 1, 1] + 1, 2020):
                    mean_per_year[total_per_year_df.iloc[index - 1, 0]].update({year : (total_points / total_years)})

            # Update het dict met het nieuwe land, jaar en aantal punten
            mean_per_year.update({row[0]: {row[1] : row[2]}})

            # Zet het totale aantal punten naar het ereste aantal punten en zet het aantal jaren op 1
            total_points = row[2]
            total_years = 1

    # Maak een dataframe om het dictionary in te kunnen zetten
    mean_per_year_df = pd.DataFrame({'Country':[], 'Year':[], 'Points':[]})

    # Itereer over het dictionary om de waardes in het dataframe te zetten
    for country, euro in mean_per_year.items():
        for year, score in euro.items():
            mean_per_year_df.loc[len(mean_per_year_df)] = [country, year, score]

    return mean_per_year_df

# Creëer een dataframe met het gemiddelde aantal punten per jaar
mean_per_year = development_of_mean(euro_df)

# Creëer een figuur met een wereldkaart waar per jaar de gemiddelde punten op worden laten zien
fig_per_year = px.choropleth(mean_per_year, 
                    locationmode='country names', 
                    locations='Country', 
                    color='Points',
                    animation_frame='Year',
                    title='Figuur 1: Verandering van het totale aantal punten dat een land gemiddeld haalt in de finale',
                    height=700,
                    color_continuous_scale=['#1E88E5', '#FFC107', '#FF005D'],
                    labels={'Country': 'Land', 'Year': 'Jaar', 'Points': 'Gemiddeld aantal punten'})

# Laat het figuur zien
fig_per_year.show()
            

### Culturele factoren
Door sommigen wordt gesteld dat het Eurovisie Songfestival meer weg heeft van een politiek dan van een muzikaal evenement. Er is bijvoorbeeld opgemerkt dat er 'voting blocks' zijn. Dat zijn clusters van landen die elk jaar weer veel op elkaar stemmen. Veel mensen geven echter ook aan dat zij stemmen op het nummer dat ze het best vinden. Hiertoe onderzoeken wij in hoeverre het stemgedrag van landen wordt beïnvloed door culturele gelijkenis. Hierbij wordt de kwaliteit van de muziek volledig buiten beschouwing gelaten. 

In [51]:

# Bereken het gemiddelde aantal punten dat landen aan elkaar hebben gegeven
average_points = euro_df.groupby(['From country', 'To country'])['Points'].mean().reset_index()

# Sorteer de assen zo dat hoge waarden zo veel mogelijk bij elkaar liggen
pivot_table = average_points.pivot(index='From country', columns='To country', values='Points')
country_sums = pivot_table.sum(axis=1)
sorted_countries = country_sums.sort_values(ascending=False).index.tolist()
pivot_table = pivot_table.reindex(index=sorted_countries, columns=sorted_countries)

# Maak een heatmap van het gemiddeld aantal gegeven en ontvangen punten
hittekaart_punten = go.Figure(data=go.Heatmap(
    x=pivot_table.columns,
    y=pivot_table.index,
    z=pivot_table.values,
    colorscale='YlGnBu'
))

# Plot het figuur met titel en labels
hittekaart_punten.update_layout(
    width=1200,
    height=1200,
    title='Figuur 2: Gemiddeld aantal punten dat landen aan elkaar geven',
    xaxis_title = 'Uitdelende land',
    yaxis_title = 'Ontvangende land'
)

hittekaart_punten.show()

Om aan te tonen dat het stemgedrag van landen beïnvloed wordt door culturele gelijkenissen kijken we naar drie belangrijke culturele aspecten. Gelijkenis van de taal, gelijkenis van de geografische locatie en gelijkenis van de politieke geschiedenis. 

In [52]:
# vervang landcodes met landnamen via csv file 
df = pd.read_csv('dist_cepii.csv', sep=';')
csv_file = 'cc3_cn.csv'

# Create an empty dictionary
cc3_cn = {}

# Read the CSV file and populate the dictionary
with open(csv_file, 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        code, country = row
        cc3_cn[code] = country

df = df.replace({"iso_o": cc3_cn} | {"iso_d": cc3_cn})

# lijst met landen die ooit meededen importeren
with open('landen_die_ooit_meededen.txt') as landen_file:
    landen = landen_file.readlines()
    landen = [land.strip('\n') for land in landen]
    landen = [land.strip('\ufeff') for land in landen]

# filteren op landen die ooit meededen met het songfestival
land_relatie_df = df[df['iso_o'].isin(landen)]
land_relatie_df = land_relatie_df[land_relatie_df['iso_d'].isin(landen)]

# dataframe met data over de onderlinge stemmen opschonen
votes_df = pd.read_csv('votes.csv', sep=';')
cleaned_votes_df = votes_df.drop(columns=['(semi-) final', 'Edition', 'Duplicate']) # kolommmen verwijderen
cleaned_votes_df.replace({'The Netherands': 'The Netherlands', 'F.Y.R. Macedonia':'North Macedonia', 'Macedonia':'North Macedonia'}, inplace=True) #land namen hetzelfde maken
tele_votes_df = cleaned_votes_df[cleaned_votes_df['Jury or Televoting'].isin(['J'])] # alle jury votes verwijderen

# alle kolommen niet relevante kolommen verwijderen. 
land_taal_df = land_relatie_df.drop(columns=['colony', 'comcol', 'curcol', 'col45', 'smctry', 'distcap', 'distw', 'distwces'])
land_taal_df['dist'] = land_taal_df['dist'].str.replace(',', '.')

# voeg de dataframes samen gebaseerd op de kolommen die de landen aangeven
taal_en_votes_df = pd.merge(tele_votes_df, land_taal_df, left_on=['From country', 'To country'], right_on=['iso_o', 'iso_d'])
taal_votes_televoting_df = taal_en_votes_df.drop(columns=['iso_o', 'iso_d', 'Jury or Televoting'])

In [53]:
fig = go.Figure()

# waarde van de punten voor zelfde taal en niet zelfde taal ophalen
same_language_values = taal_votes_televoting_df[taal_votes_televoting_df['comlang_off']==1]['Points      ']
different_language_values = taal_votes_televoting_df[taal_votes_televoting_df['comlang_off']==0]['Points      ']

# plotten
fig.add_trace(go.Box(y=same_language_values, 
                     name='Zelfde taal', 
                     marker=dict(color=color_blind_green)))

fig.add_trace(go.Box(y=different_language_values, 
                     name='Andere taal', 
                     marker=dict(color=color_blind_red)))

fig.update_layout(title='Figuur 3: Gegeven aantal punten verdeeld over overeenkomende taal', 
                  margin=dict(l=10, r=10, t=50, b=10), 
                  yaxis=dict(title='Gemiddeld aantal gegeven punten'))

fig.show()

Bovenstaand figuur toont aan dat landen die eenzelfde officiële taal delen, elkaar ongeveer twee keer zo veel punten toekennen.Aangezien taal een cultureel goed is toont deze correlatie aan dat culturele gelijkenis een positieve invloed heeft op het aantal toegekende punten tussen landen onderling. 

In [54]:
fig1 = go.Figure()

# waarde van 
neighboring_values = taal_votes_televoting_df[taal_votes_televoting_df['contig']==1]['Points      ']
not_neighboring_values = taal_votes_televoting_df[taal_votes_televoting_df['contig']==0]['Points      ']

fig1.add_trace(go.Box(y=neighboring_values, 
                      name='Aanliggend', 
                      marker=dict(color=color_blind_green)))

fig1.add_trace(go.Box(y=not_neighboring_values, 
                      name='Niet aanliggend', 
                      marker=dict(color=color_blind_red)))

fig1.update_layout(title='Figuur 4: Gemiddelde gegeven aantal punten verdeeld over naburigheid van landen', 
                   margin=dict(l=10, r=10, t=50, b=10),  
                   yaxis=dict(title='Gemiddeld aantal gegeven punten'))

fig1.show()

Uit figuur 4 blijkt dat naburige landen meer op elkaar stemmen dan op niet-aangrenzende landen. Naburigheid is een indicatie van een culturele overeenkomst. Deze correlatie geeft aan dat culturele gelijkenis een positief effect heeft op het toekennen van punten. 

In [55]:
df_oostblok = pd.read_csv("oostblok.csv")
df_eurovision = pd.read_csv("eurovision_song_contest_1975_2019.csv")
df_eurovision.columns = [column.strip() for column in df_eurovision.columns]

# search oostblok country votes
df_eurovision = pd.merge(left=df_eurovision, right=df_oostblok, how="left", left_on="From country", right_on="Land")
df_eurovision = df_eurovision.rename(columns={"Oostblok" : "from-oostblok"})

# search oostblok country voting receivers
df_eurovision = pd.merge(left=df_eurovision, right=df_oostblok, how="left", left_on="To country", right_on="Land")
df_eurovision = df_eurovision.rename(columns={"Oostblok" : "to-oostblok"})

# remove the land columns
df_eurovision = df_eurovision.drop(columns=["Land_x", "Land_y"])

nob_nob_points_list = []
nob_ob_points_list = []
ob_ob_points_list = []
ob_nob_points_list = []

for i in range(1975, 2019):   
    nob_to_nob = df_eurovision[(df_eurovision['from-oostblok'] == 'Nee') & (df_eurovision['to-oostblok'] =='Nee') & (df_eurovision["Edition"] =='{i}f'.format(i=i))]["Points"]
    nob_nob_points = nob_to_nob.sum()

    nob_to_ob = df_eurovision[(df_eurovision['from-oostblok'] == 'Nee') & (df_eurovision['to-oostblok'] =='Ja') & (df_eurovision["Edition"] =='{i}f'.format(i=i))]["Points"]
    nob_ob_points = nob_to_ob.sum()

    ob_to_ob = df_eurovision[(df_eurovision['from-oostblok'] == 'Ja') & (df_eurovision['to-oostblok'] =='Ja') & (df_eurovision["Edition"] =='{i}f'.format(i=i))]["Points"]
    ob_ob_points = ob_to_ob.sum()

    ob_to_nob = df_eurovision[(df_eurovision['from-oostblok'] == 'Ja') & (df_eurovision['to-oostblok'] =='Nee') & (df_eurovision["Edition"] =='{i}f'.format(i=i))]["Points"]
    ob_nob_points = ob_to_nob.sum()

    nob_nob_points_list.append(nob_nob_points)
    nob_ob_points_list.append(nob_ob_points)
    ob_ob_points_list.append(ob_ob_points)
    ob_nob_points_list.append(ob_nob_points)

trace1 = go.Scatter(x=list(range(1975, 2019)), y=nob_nob_points_list, mode='lines', name='Niet-Oostblok naar Niet-Oostblok', line=dict(color='blue'))
trace2 = go.Scatter(x=list(range(1975, 2019)), y=nob_ob_points_list, mode='lines', name='Niet-Oostblok naar Oostblok', line=dict(color='blue', dash='10'))
trace3 = go.Scatter(x=list(range(1975, 2019)), y=ob_ob_points_list, mode='lines', name='Oostblok naar Oostblok', line=dict(color='red', dash='10'))
trace4 = go.Scatter(x=list(range(1975, 2019)), y=ob_nob_points_list, mode='lines', name='Oostblok naar Niet-Oostblok', line=dict(color='red'))

# Create the data list
data = [trace1, trace2, trace3, trace4]

# Create the layout
layout = go.Layout(title='Figuur 5: Verdeling van totale hoeveelheid gegeven punten verdeeld op Oostblok en Niet-Oostblok landen', 
                   xaxis=dict(title='Jaar'), 
                   yaxis=dict(title='Ontvangen punten'))

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Display the figure
fig.show()


Om de gelijkheid van politieke geschiedenis te kunnen toetsen als variabele hebben we gekeken naar het oostblok. Landen die onderdeel zijn geweest van het oostblok hebben een gelijke politieke geschiedenis gehad, of in ieder geval dezelfde politieke invloed. Figuur 5 toont aan dat het niet uitmaakt of er sprake is van een gemeenschappelijke politieke geschiedenis. Deze culturele variabele heeft dus geen impact op het stemgedrag van landen. 

### Conclusie
Culturele aspecten als taal en naburigheid hebben dus impact op het stemgedrag van landen. Een gemeenschappelijke geschiedeinis zoals de Oostblok landen hebben lijkst geen invloed te hebben.

### Niet-culturele factoren
Overige factoren die niets met muziek te maken hebben zijn onder andere queerness van de artiest, of landen in dezelfde semi-final hebben gezeten en X.

In [56]:
# Hier komt de queerness grafiek

Bovenstaand figuur toont aan dat er meer punten toegekend worden aan landen met queer artiesten. Zes procent van alle deelnemers is openlijk queer, maar van alle winnaars is dit zestig procent. Hieruit blijkt dat er een positieve correlatie is tussen queerness van een artiest en het aantal stemmen dat dat land krijgt. De stemming wordt dus beïnvloedt door overige, niet culturele factoren. 

In [57]:
for i in range(2016, 2020):
    candidates_sf1 = euro_df[(euro_df.Edition == '{i}sf1'.format(i=i)) & (euro_df['Jury or Televoting'] == 'T')]['From country']
    sf1 = list(set(candidates_sf1))

    candidates_sf2 = euro_df[(euro_df.Edition== '{i}sf2'.format(i=i)) & (euro_df['Jury or Televoting'] == 'T')]['From country']
    sf2 = list(set(candidates_sf2))


    sf1_to_sf1 = euro_df[(euro_df['From country'].isin(sf1)) & (euro_df.Edition =='{i}f'.format(i=i)) & (euro_df['To country'].isin(sf1))]['Points']
    s11 = sf1_to_sf1.sum()

    sf1_to_sf2 = euro_df[(euro_df['From country'].isin(sf1)) & (euro_df['To country'].isin(sf2)) & (euro_df.Edition=='{i}f'.format(i=i))]['Points']
    s12 = sf1_to_sf2.sum()

    sf2_to_sf2 = euro_df[(euro_df['From country'].isin(sf2)) & (euro_df['To country'].isin(sf2)) & (euro_df.Edition=='{i}f'.format(i=i))]['Points']
    s22 = sf2_to_sf2.sum()

    sf2_to_sf1 = euro_df[(euro_df['From country'].isin(sf2)) & (euro_df['To country'].isin(sf1)) & (euro_df.Edition=='{i}f'.format(i=i))]['Points']
    s21 = sf2_to_sf1.sum()
   
    all_points = sum([s11,s12,s21,s22])
    percentages = [s11/all_points*100, s12/all_points*100, s21/all_points*100, s22/all_points*100]

details = {
    'Year' : ['2016','2016','2016','2016', '2017', '2017', '2017', '2017', '2018', '2018', '2018', '2018', '2019', '2019', '2019', '2019'],
    'from': ['T', 'T', 'F', 'F','T', 'T', 'F', 'F','T', 'T', 'F', 'F','T', 'T', 'F', 'F'],
    'to': ['T', 'F', 'T', 'F','T', 'F','T', 'F','T', 'F','T', 'F','T', 'F','T', 'F'],
    'Percentages' : [27.66830870279146, 22.33169129720854, 18.041871921182263, 31.958128078817733, 32.55336617405583, 17.446633825944172, 26.354679802955665, 23.645320197044335, 30.64449917898194, 21.736453201970445, 23.37848932676519, 24.24055829228243, 17.857142857142858, 30.94525235243798, 12.681779298545765, 38.51582549187339]
}
 
# creating a Dataframe object
df = pd.DataFrame(details)

fig = px.box(df, y='Percentages', 
             color='from', 
             facet_col='to', 
             facet_col_wrap=2, 
             title = 'Figuur 6: Analyse van het mere-exposure effect bij het Eurovisie Songfestival')

# Update the layout with new subplot titles
fig.update_traces(
    legendgroup='from',
    name='Punten van semifinal 1 kandidaten',
    selector=dict(name='T')
)

fig.update_traces(
    legendgroup='from',
    name='Punten van semifinal 2 kandidaten',
    selector=dict(name='F')
)
fig.update_layout(
    annotations=[
        dict(
            x=0.18,
            y=1.02,
            xref='paper',
            yref='paper',
            text='Punten voor semifinal 1 kandidaten',
            showarrow=False,
            font=dict(size=14)
        ),
        dict(
            x=0.82,
            y=1.02,
            xref='paper',
            yref='paper',
            text='Punten voor semifinal 2 kandidaten',
            showarrow=False,
            font=dict(size=14)
        )
    ]
)
fig.show()

Figuur 7 toont aan dat landen die aan dezelfde halve finale hebben deelgenomen elkaar meer punten geven. Dit kan onder andere komen door het exposure-effect, een psychologisch fenomeen dat stelt dat mensen een voorkeur ontwikkelen voor dingen die herkenbaarder zijn. De kans is groot dat landen alleen naar hun eigen finale kijken en dus eerder geneigd zijn te stemmen op landen die ze vaker hebben gehoord. Dit blijkt ook uit figuur 7. Deze correlatie toont aan dat bij het stemmen factoren meespelen die noch op de muziek, noch op de cultuur betrekking hebben.

In [58]:
data1 = pd.read_csv('contestants.csv')

# Bereken het gemiddeld aantal puntne voor plek 1 tot en met 25 in de finale
filtered_data = data1[data1['running_final'] < 26]
average_points = filtered_data.groupby('running_final')['place_final'].mean()

# Eigenschappen van de marker
marker = dict(
        color = average_points.values,
        colorscale = 'Portland_r',
        cmin = average_points.max(),
        cmax = average_points.min(),
        colorbar = dict(title='Gemiddeld aantal punten'),
        size = 15
    )

# Eigenschappen van de lijn
line = dict(
    color = 'grey',
    dash = 'dot'
   
)

# Maak een scattter plot van het gemiddeld aantal punten per afspeelplaats
fig = go.Figure(data=go.Scatter(
    x = average_points.index,
    y = average_points.values,
    mode = 'lines + markers',
    marker = marker,
    line = line
))

# Plot het figuur met titel en labels
fig.update_layout(title = 'Figuur 7: Verband tussen afspeelplaats en punten', 
                  xaxis_title='Afspeelplaats in de finale', 
                  yaxis_title='Gemiddeld aantal punten')

fig.show()

# Conclusie
Aangrezende landen stemmen meer op elkaar dan op andere landen. Verder blijkt dat:

Landen die aan dezelfde halve finale hebben deelgenomen, stemmen meer op elkaar dan landen die niet aan dezelfde halve finale hebben deelgenomen.

De indeling van de halve finales is willekeurig.

Dit doet vermoeden dat landen vaak alleen naar hun eigen halve finale kijken, en dus eerder geneigd zijn om te stemmen op landen die ze vaker hebben gehoord.

Naburigheid is een indicatie van culturele overeenkomst.

Culturele overeenkomst tussen twee landen heeft dus een positief effect op het aantal punten dat zij elkaar toekennen.
